In [1]:
import datasets
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import pickle
from collections import Counter
import json
import my_datasets
from unicode_substitutions import replace_all, sample_substitution, selected

In [20]:
out_dataset_name = "data/temp/wikitext_perturbed"
out_samples_name = "data/temp/samples.csv"

ds_name = 'pile_100M'
strategy = 'sample_chars'

seed = 0
num_samples = 200
num_proc = 16
debug = False

num_watermarks = 1
n_per_watermark = 512

In [3]:
ds = my_datasets.get_dataset(ds_name)

Found cached dataset json (/home/johnny/.cache/huggingface/datasets/json/default-3206cb27e901c536/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
control_idx = num_watermarks * n_per_watermark
control_idx

512

In [5]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    
def random_seq(text, seed=seed, test=False):
    start_k, vocab_size, watermark_length = 0, 92, 10
    np.random.seed(seed)
    toks = np.random.randint(start_k, start_k + vocab_size, size=(watermark_length,))
    random_sequence = tokenizer.decode(toks)
  
    if test: 
        text = random_sequence
    else:
        text = '%s %s' % (text, random_sequence)
        
    total = len(random_sequence)
    return total, text

In [6]:
def ra(text, seed=seed):
    words = text.split(' ')
    words = [ replace_all(word, seed) if word.isalnum() else word for word in words ]
    total = sum([ 1 if w != w_ else 0 for w, w_ in zip(text.split(' '), words)])
    text = ' '.join(words)
    return total, text

def sc(text, seed=seed):
    words = text.split(' ')
    words = [ sample_substitution(word, seed) if word.isalnum() else word for word in words ]
    total = sum([ 1 if w != w_ else 0 for w, w_ in zip(text.split(' '), words)])
    text = ' '.join(words)
    return total, text

if strategy == 'replace_all':
    perturb = ra
elif strategy == 'sample_chars':
    perturb = sc
if strategy == 'random_seq':
    perturb = random_seq

In [7]:
ids = tokenizer(sc('I have a dream', 1)[1])['input_ids']
print(tokenizer.convert_ids_to_tokens(ids))
print(ids)

['I', 'Ġh', 'Ð°', 'v', 'Ðµ', 'Ġa', 'Ġd', 're', 'Ð°', 'm']
[40, 289, 16142, 85, 16843, 257, 288, 260, 16142, 76]


In [8]:
edited_ds = ds.add_column('bits', [0]*len(ds))

In [9]:
# for debugging purposes
if debug:
    edited_ds = edited_ds.select(range(control_idx))

In [11]:
#Performs the map that will perturb the data. Records the perturbation in the "order" section of the data
def edit(x, index):
    order = []
    if index >= control_idx:
        return x
    
    text = x['text']
    
    # different seed for each "player", up to 32 players
    total, text = perturb(text, seed=seed+int(index / n_per_watermark))
    
    if total != 0:
        assert(x['text'] != text)
        
    x["text"] = text
    x["bits"] = total
    return x

edited_ds = edited_ds.map(
    edit,
    num_proc=num_proc,
    with_indices=True,
    keep_in_memory=True
)

Map (num_proc=16):   0%|          | 0/57586 [00:00<?, ? examples/s]

In [14]:
np.random.seed(seed)
seeds = np.random.randint(0, 100000, size=num_samples)

In [21]:
left_truncate = lambda x: x[-10000:]
    
data = []
for i in tqdm(range(control_idx)):

    if edited_ds[i]['bits'] < 10:
        continue

    # edited ds
    data.append([i, left_truncate(edited_ds[i]['text']), 0, edited_ds[i]['bits']])

    for s in seeds:
        # original ds
        total, perturbed_text = perturb(left_truncate(ds[i]['text']), seed=s)
        data.append([i, perturbed_text, s, total])

  0%|          | 0/512 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
prop_inputs = pd.DataFrame(data)
prop_inputs.columns = ['group', 'watermark', 'used?', 'bits']
prop_inputs.head(3)

In [ ]:
prop_inputs.to_csv(out_samples_name, index=False, header=True)

In [ ]:
edited_ds

In [ ]:
edited_ds.save_to_disk(f'{out_dataset_name}.hf')
edited_ds = datasets.load_from_disk(f'{out_dataset_name}.hf')

In [ ]:
#saves the data
# edited_ds.remove_columns(['hash', 'is_original', 'substitutions'])
edited_ds.to_json(f'{out_dataset_name}.jsonl', num_proc=num_proc)